# 智能考试助手 - Jupyter Notebook版本

## 项目概述
这是一个基于AI的考试解答助手，支持多种AI API：
- 智谱AI BigModel API
- DeepSeek API
- 火山引擎API

## 功能特性
- 自动浏览器连接和页面导航
- 智能题目提取和选项识别
- AI答案生成和显示
- 交互式操作界面
- 结果保存功能

## 1. 环境检查和依赖安装

In [ ]:
# 检查并安装必要的依赖包
import sys
import subprocess

def install_package(package_name):
    try:
        __import__(package_name)
        print(f"✅ {package_name} 已安装")
    except ImportError:
        print(f"📦 正在安装 {package_name}...")
        try:
            # 使用国内镜像源加速安装
            subprocess.check_call([
                sys.executable, "-m", "pip", "install", 
                "--timeout", "300", "--retries", "3",
                "-i", "https://pypi.tuna.tsinghua.edu.cn/simple",
                "--trusted-host", "pypi.tuna.tsinghua.edu.cn",
                package_name
            ])
            print(f"✅ {package_name} 安装成功")
        except subprocess.CalledProcessError as e:
            print(f"❌ {package_name} 安装失败: {e}")

# 需要安装的包列表
required_packages = [
    "zhipuai",
    "openai",
    "drissionpage",
    "selenium"
]

print("🔍 检查依赖包安装状态...")
for package in required_packages:
    install_package(package)

print("\n✅ 依赖包检查完成")

# 导入所需的库
try:
    import os
    import time
    import json
    from DrissionPage import ChromiumPage
    from zhipuai import ZhipuAI
    from openai import OpenAI
    print("✅ 所有库导入成功")
except ImportError as e:
    print(f"❌ 库导入失败: {e}")
    print("请重启Jupyter内核后重试")

API

In [ ]:
# API密钥配置
ZHIPU_API_KEY = "58c16dcd18a243788f32cafa7928d64f.Cglq3ehd3s7agRBe"
DEEPSEEK_API_KEY = "sk-7c672611a0414600b47dd79f40090a9b"

# 初始化API客户端
zhipu_client = ZhipuAI(api_key=ZHIPU_API_KEY)
deepseek_client = OpenAI(
    api_key=DEEPSEEK_API_KEY,
    base_url="https://api.deepseek.com"
)

print("✅ API客户端初始化完成")

生成函数

In [ ]:
def call_zhipu_api(question, options, model="glm-4.6"):
    """调用智谱AI API生成答案"""
    prompt = f"""
请分析以下考试题目并选择最合适的答案：

题目：{question}
选项：
{chr(10).join([f'{chr(65+i)}. {option}' for i, option in enumerate(options)])}

请只返回选项字母（如A、B、C、D），不要包含其他内容。
"""
    
    try:
        response = zhipu_client.chat.completions.create(
            model=model,
            messages=[{
                "role": "user",
                "content": prompt
            }]
        )
        answer = response.choices[0].message.content.strip()
        return answer
    except Exception as e:
        print(f"❌ 智谱AI API调用失败: {e}")
        return None

def call_deepseek_api(question, options, model="deepseek-chat"):
    """调用DeepSeek API生成答案"""
    prompt = f"""
请分析以下考试题目并选择最合适的答案：

题目：{question}
选项：
{chr(10).join([f'{chr(65+i)}. {option}' for i, option in enumerate(options)])}

请只返回选项字母（如A、B、C、D），不要包含其他内容。
"""
    
    try:
        response = deepseek_client.chat.completions.create(
            model=model,
            messages=[{
                "role": "user",
                "content": prompt
            }]
        )
        answer = response.choices[0].message.content.strip()
        return answer
    except Exception as e:
        print(f"❌ DeepSeek API调用失败: {e}")
        return None

自动化

In [ ]:
def init_browser():
    """初始化浏览器"""
    try:
        page = ChromiumPage()
        print("✅ 浏览器初始化成功")
        return page
    except Exception as e:
        print(f"❌ 浏览器初始化失败: {e}")
        return None

def navigate_to_exam_page(page, url):
    """导航到考试页面"""
    try:
        page.get(url)
        print(f"✅ 已导航到: {url}")
        return True
    except Exception as e:
        print(f"❌ 页面导航失败: {e}")
        return False

提取函数

In [ ]:
def extract_question_and_options(page):
    """从页面提取题目和选项"""
    try:
        # 这里需要根据实际的考试页面结构来调整选择器
        # 示例选择器，需要根据实际页面调整
        question_element = page.ele('@class=question')
        options_elements = page.eles('@class=option')
        
        if question_element and options_elements:
            question = question_element.text
            options = [option.text for option in options_elements[:4]]  # 取前4个选项
            
            print(f"📝 题目: {question}")
            for i, option in enumerate(options):
                print(f"   {chr(65+i)}. {option}")
            
            return question, options
        else:
            print("❌ 无法找到题目或选项元素")
            return None, None
    except Exception as e:
        print(f"❌ 题目提取失败: {e}")
        return None, None

交互式

In [ ]:
def interactive_exam_assistant():
    """交互式考试助手主函数"""
    print("🎯 智能考试助手启动...")
    
    # 选择AI服务
    print("\n🤖 请选择AI服务：")
    print("1. 智谱AI (GLM-4.6)")
    print("2. DeepSeek (deepseek-chat)")
    
    choice = input("请输入选择 (1或2): ")
    
    if choice == "1":
        ai_service = "zhipu"
        print("✅ 已选择智谱AI")
    elif choice == "2":
        ai_service = "deepseek"
        print("✅ 已选择DeepSeek")
    else:
        print("❌ 无效选择，默认使用智谱AI")
        ai_service = "zhipu"
    
    # 输入考试页面URL
    url = input("\n🌐 请输入考试页面URL: ")
    
    # 初始化浏览器
    page = init_browser()
    if not page:
        return
    
    # 导航到考试页面
    if not navigate_to_exam_page(page, url):
        return
    
    # 开始处理题目
    question_count = 1
    results = []
    
    while True:
        print(f"\n📚 正在处理第 {question_count} 题...")
        
        # 提取题目和选项
        question, options = extract_question_and_options(page)
        
        if not question or not options:
            print("❌ 无法提取题目，请检查页面结构")
            break
        
        # 调用AI API
        if ai_service == "zhipu":
            answer = call_zhipu_api(question, options)
        else:
            answer = call_deepseek_api(question, options)
        
        if answer:
            print(f"🤖 AI答案: {answer}")
            results.append({
                "question_number": question_count,
                "question": question,
                "options": options,
                "ai_answer": answer
            })
        else:
            print("❌ AI未能生成答案")
        
        # 等待用户操作
        input("\n⏳ 请手动选择答案并点击下一题，完成后按回车键继续...")
        
        question_count += 1
        
        # 检查是否还有更多题目
        if question_count > 10:  # 限制处理题目数量
            print("⚠️ 已达到最大题目处理数量")
            break
    
    # 保存结果
    save_results(results, ai_service)
    
    print("\n🎉 考试助手处理完成！")
    print(f"📊 共处理了 {len(results)} 道题目")

In [ ]:
def save_results(results, ai_service):
    """保存处理结果"""
    timestamp = time.strftime("%Y%m%d_%H%M%S")
    filename = f"exam_results_{ai_service}_{timestamp}.json"
    
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(results, f, ensure_ascii=False, indent=2)
    
    print(f"💾 结果已保存到: {filename}")
    
    # 同时保存为文本格式
    txt_filename = f"exam_results_{ai_service}_{timestamp}.txt"
    with open(txt_filename, 'w', encoding='utf-8') as f:
        f.write(f"AI考试助手结果报告 ({ai_service})\n")
        f.write(f"生成时间: {time.strftime('%Y-%m-%d %H:%M:%S')}\n\n")
        
        for result in results:
            f.write(f"第 {result['question_number']} 题:\n")
            f.write(f"题目: {result['question']}\n")
            for i, option in enumerate(result['options']):
                f.write(f"{chr(65+i)}. {option}\n")
            f.write(f"AI答案: {result['ai_answer']}\n\n")
    
    print(f"📝 文本报告已保存到: {txt_filename}")

运行

In [ ]:
# 运行交互式考试助手
interactive_exam_assistant()

测试

In [ ]:
# 测试智谱AI API
def test_zhipu_api():
    print("🧪 测试智谱AI API...")
    
    test_question = "2 + 2 等于多少？"
    test_options = ["3", "4", "5", "6"]
    
    answer = call_zhipu_api(test_question, test_options)
    
    if answer:
        print(f"✅ 智谱AI测试成功，答案: {answer}")
    else:
        print("❌ 智谱AI测试失败")

# 测试DeepSeek API
def test_deepseek_api():
    print("🧪 测试DeepSeek API...")
    
    test_question = "中国的首都是哪里？"
    test_options = ["上海", "北京", "广州", "深圳"]
    
    answer = call_deepseek_api(test_question, test_options)
    
    if answer:
        print(f"✅ DeepSeek测试成功，答案: {answer}")
    else:
        print("❌ DeepSeek测试失败")

# 运行测试
test_zhipu_api()
test_deepseek_api()